In [1]:
import os
os.chdir(r"C:\Users\p1a2r\OneDrive\Desktop\Git Hub Projects\Text-Summarizer")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelEvaluationConfig:
    root_dir: Path 
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name)
        
        return model_evaluation_config

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device= torch.device("cuda" if torch.cuda.is_available() else "cpu"),
                                column_text="article",
                                column_summary="highlights"):
        article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load('rouge')
        
        score = self.calculate_metric_on_test_ds(
            dataset_samsum['test'], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary')
        
        rouge_dict = {}
        for rn in rouge_names:
            score_value = score[rn]
            if hasattr(score_value, 'mid'):
                # Old format: score object with .mid.fmeasure
                rouge_dict[rn] = score_value.mid.fmeasure
            else:
                # New format: direct float value
                rouge_dict[rn] = score_value

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [ ]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
    logger.info("Model evaluation completed successfully!")
    
except Exception as e:
    logger.error(f"Model evaluation failed: {str(e)}")
    raise e